In [4]:
exportdate = 20180327
projectname = 'repract'

In [5]:
import pandas as pd, re

First, only vote counts merged with tags

In [9]:
counts_only = pd.read_csv(f'../../analysis/{exportdate}{projectname}_evaluation_counts.csv')
counts_only.head()

,PaperID,Essential,Worthwhile,Unimportant,Unwise,ZeroRating,NotShown,PosRatings,NegRatings,TotalRatings
0,248,2,5,6,0,0,141,7,6,13
1,100,7,5,0,0,1,141,12,0,12
2,39,1,6,4,0,0,143,7,4,11
3,104,2,5,4,0,1,142,7,4,11
4,108,1,9,0,1,0,143,10,1,11


In [42]:
likert_cols = ['Essential', 'Worthwhile', 'Unimportant', 'Unwise']
tendency_cols = ['PosRatings', 'NegRatings'] # NB: ZeroRating is not included in TotalRatings!
total_cols = ['TotalRatings']

In [12]:
tags = pd.read_csv('../../analysis/papersummaries_tagged_what.csv').fillna('')
tags.head()

,PaperID,Tag,level_1,level_2,level_3,level_4
0,1,what:phase:management,what,phase,management,
1,1,what:documentation:artifacts,what,documentation,artifacts,
2,1,what:challenge:content_traceability,what,challenge,content,traceability
3,1,what:challenge:process_automation,what,challenge,process,automation
4,2,what:information:functional,what,information,functional,


In [25]:
levels = ['level_1', 'level_2', 'level_3', 'level_4']

In [112]:
df = tags.merge(counts_only)
df.head()

,PaperID,Tag,level_1,level_2,level_3,level_4,Essential,Worthwhile,Unimportant,Unwise,ZeroRating,NotShown,PosRatings,NegRatings,TotalRatings
0,1,what:phase:management,what,phase,management,,1,3,1,0,0,149,4,1,5
1,1,what:documentation:artifacts,what,documentation,artifacts,,1,3,1,0,0,149,4,1,5
2,1,what:challenge:content_traceability,what,challenge,content,traceability,1,3,1,0,0,149,4,1,5
3,1,what:challenge:process_automation,what,challenge,process,automation,1,3,1,0,0,149,4,1,5
4,2,what:information:functional,what,information,functional,,2,1,1,0,1,149,3,1,4


In [83]:
def percentages(df, base_col='TotalRatings'):
    newdf = pd.DataFrame(df, copy=True)
    for col in newdf.columns:
        if col != base_col:
            newdf[col] = newdf[col] / newdf[base_col]
    #newdf['RankAid'] = newdf['PosRatings'] * newdf['TotalRatings']
    return newdf

In [84]:
absolute = df[levels + tendency_cols + total_cols
  ].groupby(levels).sum().sort_values('PosRatings', ascending=False)
absolute.head(10)

PosRatings  NegRatings  \
level_1 level_2       level_3         level_4                                
what    phase         management                           249         109   
        challenge     process         automation           158          90   
                      content         traceability         153          74   
        phase         elicitation                          150          67   
        documentation naturallanguage                      144          70   
        phase         evaluation                           136          53   
        challenge     process         improving            124          61   
        information   quality         security             100          32   
        challenge     context         uncertainty           96          37   
        phase         specification                         95          34   

                                                    TotalRatings  
level_1 level_2       level_3         level_4                     
what    phase         management                             358  
        challenge     process         automation             248  
                      content         traceability           227  
        phase         elicitation                            217  
        documentation naturallanguage                        214  
        phase         evaluation                             189  
        challenge     process         improving              185  
        information   quality         security               132  
        challenge     context         uncertainty            133  
        phase         specification                          129

In [85]:
relative = percentages(df[levels + tendency_cols + total_cols
  ].groupby(levels).sum(), 'TotalRatings').sort_values('PosRatings', ascending=False)
relative.head(10)

PosRatings  NegRatings  \
level_1 level_2       level_3      level_4                                  
what    domain        systemclass  complex           1.000000    0.000000   
        documentation prototypes                     1.000000    0.000000   
        domain        organization outsourced        1.000000    0.000000   
        challenge     content      consistency       0.875000    0.125000   
        domain        systemclass  safetycritical    0.842105    0.157895   
        information   architecture                   0.838710    0.161290   
        domain        systemclass  embedded          0.833333    0.166667   
        information   rules                          0.823529    0.176471   
                      quality      safety            0.821429    0.178571   
                      scenarios                      0.821429    0.178571   

                                                   TotalRatings  
level_1 level_2       level_3      level_4                       
what    domain        systemclass  complex                    7  
        documentation prototypes                              6  
        domain        organization outsourced                 4  
        challenge     content      consistency               48  
        domain        systemclass  safetycritical            19  
        information   architecture                           31  
        domain        systemclass  embedded                   6  
        information   rules                                  17  
                      quality      safety                    28  
                      scenarios                              28

In [86]:
relative.tail(20)

PosRatings  NegRatings  \
level_1 level_2       level_3        level_4                                  
what    region        country        finland           0.666667    0.333333   
        documentation userstories                      0.666667    0.333333   
        information   functional                       0.653846    0.346154   
        challenge     people         skills            0.641026    0.358974   
                      process        automation        0.637097    0.362903   
        domain        sector         media             0.625000    0.375000   
        region        country        china             0.600000    0.400000   
        challenge     content        feasibility       0.600000    0.400000   
        information   systembehavior                   0.600000    0.400000   
        domain        systemclass    bi                0.600000    0.400000   
        information   quality        sustainability    0.578947    0.421053   
                                     reliability       0.571429    0.428571   
        domain        sector         supplier          0.562500    0.437500   
                                     energy            0.545455    0.454545   
        documentation goalmodels                       0.531250    0.468750   
                      businessmodels                   0.500000    0.500000   
                      statemachines                    0.500000    0.500000   
        general       research                         0.466667    0.533333   
        domain        sector         nanotechnology    0.400000    0.600000   
                                     subsea            0.166667    0.833333   

                                                     TotalRatings  
level_1 level_2       level_3        level_4                       
what    region        country        finland                    6  
        documentation userstories                              18  
        information   functional                               52  
        challenge     people         skills                    78  
                      process        automation               248  
        domain        sector         media                     32  
        region        country        china                      5  
        challenge     content        feasibility               25  
        information   systembehavior                           10  
        domain        systemclass    bi                         5  
        information   quality        sustainability            19  
                                     reliability               14  
        domain        sector         supplier                  16  
                                     energy                    11  
        documentation goalmodels                               32  
                      businessmodels                            8  
                      statemachines                             4  
        general       research                                 30  
        domain        sector         nanotechnology             5  
                                     subsea                     6

In [88]:
percentages(df[df.level_2 == 'challenge'][levels[2:] + tendency_cols + total_cols
                             ].groupby(levels[2:]).sum()
           ).sort_values(['PosRatings', 'TotalRatings'], ascending=False)

PosRatings  NegRatings  TotalRatings
level_3 level_4                                                
content consistency          0.875000    0.125000            48
process standardization      0.795699    0.204301            93
people  collaboration        0.780000    0.220000            50
process deciding             0.773585    0.226415            53
failure                      0.772727    0.227273            44
content understandability    0.763889    0.236111            72
        completeness         0.760563    0.239437            71
problem                      0.730769    0.269231            78
process prioritization       0.727273    0.272727            77
people  communication        0.725664    0.274336           113
process formalization        0.722222    0.277778            18
context uncertainty          0.721805    0.278195           133
content unambiguousness      0.721519    0.278481            79
process visualization        0.704918    0.295082            61
people  subjectivity         0.694444    0.305556            36
content all                  0.681818    0.318182            22
        traceability         0.674009    0.325991           227
context regulation           0.670330    0.329670            91
process improving            0.670270    0.329730           185
people  skills               0.641026    0.358974            78
process automation           0.637097    0.362903           248
content feasibility          0.600000    0.400000            25

In [101]:
percentages(absolute[absolute.TotalRatings >= 50]).sort_values(['PosRatings', 'TotalRatings'], ascending=False)

PosRatings  \
level_1 level_2       level_3         level_4                         
what    documentation artifacts                            0.796875   
        challenge     process         standardization      0.795699   
                      people          collaboration        0.780000   
                      process         deciding             0.773585   
        phase         analysis                             0.773333   
        challenge     content         understandability    0.763889   
        domain        organization    agile                0.761194   
        challenge     content         completeness         0.760563   
        information   quality         security             0.757576   
        documentation usecases                             0.750000   
        information   quality         all                  0.741176   
        phase         specification                        0.736434   
        challenge     problem                              0.730769   
                      process         prioritization       0.727273   
                      people          communication        0.725664   
                      context         uncertainty          0.721805   
                      content         unambiguousness      0.721519   
        phase         evaluation                           0.719577   
        challenge     process         visualization        0.704918   
        documentation featuremodels                        0.698113   
        phase         management                           0.695531   
                      elicitation                          0.691244   
        challenge     content         traceability         0.674009   
        information   quality         performance          0.673077   
        documentation naturallanguage                      0.672897   
        challenge     context         regulation           0.670330   
                      process         improving            0.670270   
        information   functional                           0.653846   
        challenge     people          skills               0.641026   
                      process         automation           0.637097   

                                                         NegRatings  \
level_1 level_2       level_3         level_4                         
what    documentation artifacts                            0.203125   
        challenge     process         standardization      0.204301   
                      people          collaboration        0.220000   
                      process         deciding             0.226415   
        phase         analysis                             0.226667   
        challenge     content         understandability    0.236111   
        domain        organization    agile                0.238806   
        challenge     content         completeness         0.239437   
        information   quality         security             0.242424   
        documentation usecases                             0.250000   
        information   quality         all                  0.258824   
        phase         specification                        0.263566   
        challenge     problem                              0.269231   
                      process         prioritization       0.272727   
                      people          communication        0.274336   
                      context         uncertainty          0.278195   
                      content         unambiguousness      0.278481   
        phase         evaluation                           0.280423   
        challenge     process         visualization        0.295082   
        documentation featuremodels                        0.301887   
        phase         management                           0.304469   
                      elicitation                          0.308756   
        challenge     content         traceability         0.325991   
        i

In [111]:
df[df.level_2 == 'challenge'][levels[2:] + likert_cols + tendency_cols + total_cols
                             ].groupby(levels[2:]).sum(
).sort_values(likert_cols[:2], ascending=False)

Essential  Worthwhile  Unimportant  Unwise  \
level_3 level_4                                                         
content traceability              59          94           67       7   
process automation                49         109           74      16   
        improving                 38          86           53       8   
people  communication             37          45           27       4   
context uncertainty               29          67           33       4   
process standardization           25          49           17       2   
context regulation                23          38           26       4   
content unambiguousness           22          35           22       0   
problem                           21          36           20       1   
content completeness              18          36           14       3   
        understandability         17          38           15       2   
people  collaboration             16          23           10       1   
process deciding                  15          26           11       1   
        prioritization            13          43           18       3   
        visualization             12          31           15       3   
content consistency               12          30            6       0   
people  skills                    11          39           21       7   
failure                           11          23            9       1   
people  subjectivity              10          15            9       2   
content feasibility                8           7            8       2   
        all                        6           9            6       1   
process formalization              6           7            5       0   

                           PosRatings  NegRatings  TotalRatings  
level_3 level_4                                                  
content traceability              153          74           227  
process automation                158          90           248  
        improving                 124          61           185  
people  communication              82          31           113  
context uncertainty                96          37           133  
process standardization            74          19            93  
context regulation                 61          30            91  
content unambiguousness            57          22            79  
problem                            57          21            78  
content completeness               54          17            71  
        understandability          55          17            72  
people  collaboration              39          11            50  
process deciding                   41          12            53  
        prioritization             56          21            77  
        visualization              43          18            61  
content consistency                42           6            48  
people  skills                     50          28            78  
failure                            34          10            44  
people  subjectivity               25          11            36  
content feasibility                15          10            25  
        all                        15           7            22  
process formalization              13           5            18

#### TODO

* vote counts with tags and paper meta and respondent meta
* integrate how? and with whom? data
* find out how to gain insight from this multi-stage-subjectivity-flawed data
* find out how to deal with lack of 'independency' due to multiple tags (papers with multiple tags contribute the same ratings stats to multiple aggregate counts)
* refine/double check/agree all taxonomies